# TODO
    - break out the fuctions into a separate file.
    - incorporate seeds into the two simulation steps as well as the sampling steps.
    - batch simulation steps 
    - figure out ancestral ne, and what scaling the mutation rate means in practice
    - figure out how to simulate unlinked loci (just run msprime many, many times??)
    - check the controlled mating is doing the expected
    - batch the analysis


In [2]:
import numpy as np
import pandas as pd
import itertools
#import scipy.spatial.distance
import matplotlib.pyplot as plt
import seaborn as sns
import os
import collections
import glob
import random
from random import shuffle

#import msprime
#import allel

#import simuPOP
#import simuPOP.utils

from pylab import rcParams
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline
sns.set_style('white')

In [3]:
#!jupyter nbconvert --to script simNe_functions_simulate.ipynb
#import simNe_functions_simulate as simNe_sim

In [4]:
#!jupyter nbconvert --to script simNe_functions_analyze.ipynb


# Start

In [5]:
batchname = 'unlinked'

In [6]:
def parse_filename(filename, get):
    basename = os.path.splitext(os.path.basename(filename))[0]
    ne, ch, fr = basename.split('_')
    Ne = np.int(ne.split('-')[1])
    Chr = np.int(ch.split('-')[1])
    Frep = np.int(fr.split('-')[1])
    if get == 'Ne':
        return(Ne)
    if get == 'Chr':
        return(Chr)
    if get == 'Frep':
        return(Frep)
    if get == 'Batchname':
        return filename.split('/')[2] # works for now
        
    #return(Ne, Chr, Frep)

#parse_filename('./share/mar14a/Ne-800_Chr-4/Ne-800_Chr-4_Frep-0.geno', get = 'Chr')

# Generate results.txt 

In [7]:
glob.glob('./share/{}/*'.format(batchname))

['./share/unlinked/Ne-50_Chr-99',
 './share/unlinked/Ne-200_Chr-99',
 './share/unlinked/Ne-800_Chr-99',
 './share/unlinked/Ne-3200_Chr-99']

In [8]:
results_list = [os.path.join(xx, 'results.txt') for xx in glob.glob('./share/{}/*'.format(batchname))]
results_dfs =[]
for res_file in results_list:
    if os.path.isfile(res_file):
        results_dfs.append(pd.read_csv(res_file, sep = '\t', header = 0, index_col=False))
results_df = pd.concat(results_dfs).reset_index(drop = True)
results_df.tail()

,file,L,S,Srep,actual_pairs_r2,overall_r2,block_r2,block_sum_squares,nonoverlap_r2,temporalF_u,temporalF_w,Patterson_f2,Hudson_fst,Jorde_Ryman_F,weir_Fst,actual_nloci_temporal
32995,./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...,4096,50,0,8386560,0.020526,0.020522,0.006739,0.021427,0.022191,0.022472,0.000455,0.001249,0.022599,0.001231,4096
32996,./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...,4096,100,0,8386560,0.010196,0.010192,0.001662,0.009359,0.013195,0.013390,0.000622,0.001703,0.013435,0.001705,4096
32997,./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...,16384,25,0,134209536,0.042193,0.042207,0.028504,0.043953,0.042770,0.043905,0.000725,0.001992,0.044392,0.001993,16384
32998,./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...,16384,50,0,134209536,0.020605,0.020606,0.006794,0.020920,0.023005,0.023229,0.000595,0.001631,0.023365,0.001643,16384
32999,./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...,16384,100,0,134209536,0.010236,0.010236,0.001676,0.010214,0.012626,0.012697,0.000492,0.001356,0.012738,0.001357,16384


In [9]:
results_df['Ne'] = results_df['file'].apply(parse_filename, get = 'Ne')
results_df['Chr'] = results_df['file'].apply(parse_filename, get = 'Chr')
results_df['Frep'] = results_df['file'].apply(parse_filename, get = 'Frep')
results_df['batchname'] = results_df['file'].apply(parse_filename, get = 'Batchname')
results_df.head()

,file,L,S,Srep,actual_pairs_r2,overall_r2,block_r2,block_sum_squares,nonoverlap_r2,temporalF_u,temporalF_w,Patterson_f2,Hudson_fst,Jorde_Ryman_F,weir_Fst,actual_nloci_temporal,Ne,Chr,Frep,batchname
0,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,16,25,0,120,0.054369,NaN,NaN,0.020408,0.227219,0.244973,0.042647,0.104578,0.260955,0.104187,16,50,99,0,unlinked
1,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,16,50,0,120,0.020503,NaN,NaN,0.016335,0.161653,0.152375,0.025525,0.066856,0.158410,0.066756,16,50,99,0,unlinked
2,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,64,25,0,2016,0.048011,0.041771,0.032271,0.044389,0.178237,0.206062,0.032914,0.084726,0.217254,0.084930,64,50,99,0,unlinked
3,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,64,50,0,2016,0.024978,0.025547,0.016680,0.026929,0.210643,0.217864,0.038039,0.099931,0.230414,0.100096,64,50,99,0,unlinked
4,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,256,25,0,32640,0.048422,0.049458,0.039473,0.045957,0.211539,0.219978,0.036515,0.091826,0.232780,0.092175,256,50,99,0,unlinked


In [10]:
results_df.fillna(0).groupby(['L', 'S', 'Ne', 'Chr']).count()

file  Srep  actual_pairs_r2  overall_r2  block_r2  \
L     S   Ne   Chr                                                      
16    25  50   99    500   500              500         500       500   
          200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
      50  50   99    500   500              500         500       500   
          200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
      100 200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
64    25  50   99    500   500              500         500       500   
          200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
      50  50   99    500   500              500         500       500   
          200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
      100 200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
256   25  50   99    500   500              500         500       500   
          200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
      50  50   99    500   500              500         500       500   
          200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
...                  ...   ...              ...         ...       ...   
1024  25  3200 99    500   500              500         500       500   
      50  50   99    500   500              500         500       500   
          200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
      100 200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
4096  25  50   99    500   500              500         500       500   
          200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
      50  50   99    500   500              500         500       500   
          200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
      100 200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
          3200 99    500   500              500         500       500   
16384 25  50   99    500   500              500         500       500   
          200  99    500   500              500         500       500   
          800  99    500   500              500         500       500   
     

In [11]:
results_df.fillna(0).groupby(['L', 'S', 'Ne', 'Chr']).max()

file  Srep  \
L     S   Ne   Chr                                                            
16    25  50   99   ./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...     0   
          200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
      50  50   99   ./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...     0   
          200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
      100 200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
64    25  50   99   ./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...     0   
          200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
      50  50   99   ./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...     0   
          200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
      100 200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
256   25  50   99   ./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...     0   
          200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
      50  50   99   ./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...     0   
          200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
...                                                               ...   ...   
1024  25  3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
      50  50   99   ./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...     0   
          200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
      100 200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
4096  25  50   99   ./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...     0   
          200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
      50  50   99   ./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...     0   
          200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked/Ne-3200_Chr-99/Ne-3200_Chr-99...     0   
      100 200  99   ./share/unlinked/Ne-200_Chr-99/Ne-200_Chr-99_F...     0   
          800  99   ./share/unlinked/Ne-800_Chr-99/Ne-800_Chr-99_F...     0   
          3200 99   ./share/unlinked

In [12]:
results_df.groupby(['L', 'S']).var() / results_df.groupby(['L', 'S']).mean()**2

Srep  actual_pairs_r2  overall_r2  block_r2  block_sum_squares  \
L     S                                                                     
16    25    NaN              0.0    0.019907       NaN                NaN   
      50    NaN              0.0    0.027341       NaN                NaN   
      100   NaN              0.0    0.020772       NaN                NaN   
64    25    NaN              0.0    0.003641  0.021019           0.097230   
      50    NaN              0.0    0.011407  0.029065           0.134027   
      100   NaN              0.0    0.004535  0.024120           0.105758   
256   25    NaN              0.0    0.002702  0.003568           0.015265   
      50    NaN              0.0    0.010195  0.011132           0.048723   
      100   NaN              0.0    0.003485  0.004372           0.018126   
1024  25    NaN              0.0    0.002575  0.002658           0.011250   
      50    NaN              0.0    0.010313  0.010389           0.045157   
      100   NaN              0.0    0.003407  0.003459           0.014270   
4096  25    NaN              0.0    0.002603  0.002605           0.010978   
      50    NaN              0.0    0.010293  0.010299           0.044631   
      100   NaN              0.0    0.003428  0.003431           0.014138   
16384 25    NaN              0.0    0.002561  0.002560           0.010772   
      50    NaN              0.0    0.010305  0.010302           0.044601   
      100   NaN              0.0    0.003386  0.003385           0.013939   

           nonoverlap_r2  temporalF_u  temporalF_w  Patterson_f2  Hudson_fst  \
L     S                                                                        
16    25        0.225621     0.548112     0.611346      1.943310    1.852690   
      50        0.250940     0.836609     0.904447      1.776897    1.680504   
      100       0.259118     0.545323     0.569871      1.353651    1.327603   
64    25        0.062320     0.451772     0.482851      1.521549    1.464838   
      50        0.070372     0.734797     0.768157      1.504121    1.440100   
      100       0.060740     0.394138     0.403492      0.946980    0.932655   
256   25        0.015570     0.436166     0.461392      1.440602    1.389220   
      50        0.024308     0.713129     0.734495      1.424066    1.373068   
      100       0.018556     0.376067     0.379145      0.885055    0.876950   
1024  25        0.005609     0.425542     0.448764      1.406633    1.356349   
      50        0.013948     0.707522     0.729812      1.409628    1.359427   
      100       0.007254     0.366282     0.368700      0.859031    0.851570   
4096  25        0.003435     0.424544     0.447606      1.401914    1.353191   
      50        0.010996     0.704420     0.726186      1.401385    1.352862   
      100       0.004500     0.364737     0.366600      0.855260    0.848518   
16384 25        0.002793     0.424565     0.447722      1.400154    1.351673   
      50        0.010564     0.704194     0.725743      1.400733    1.352209   
      100       0.003600     0.364082     0.365900      0.852642    0.845738   

           Jorde_Ryman_F  weir_Fst  actual_nloci_temporal        Ne  Chr  \
L     S                                                                    
16    25        0.659744  1.848672                    0.0  1.419533  0.0   
      50        0.958070  1.679234                    0.0  1.419533  0.0   
      100       0.583045  1.327582                    0.0  0.857715  0.0   
64    25        0.513777  1.463260                    0.0  1.419533  0.0   
      50        0.804985  1.440508                    0.0  1.419533  0.0   
      100       0.410249  0.932875                    0.0  0.857715  0.0   
256   25        0.489632  1.389011                    0.0  1.419533  0.0   
      50        0.766666  1.373381                    0.0  1.419533  0.0   
      100       0.384876  0.877079                    0.0  0.857715  0.0   
1024  25        0.475664  1.35657

In [13]:
results_df.to_csv('./share/results/{}.results.txt'.format(batchname), index = False, sep = '\t', na_rep = 'NA')

In [14]:
pair_results_list = [os.path.join(xx, 'results.pairs.txt') for xx in glob.glob('./share/{}/*'.format(batchname))]
pair_results_dfs =[]
for res_file in pair_results_list:
    if os.path.isfile(res_file):
        pair_results_dfs.append(pd.read_csv(res_file, sep = '\t', header = 0, index_col=False))
pair_results_df = pd.concat(pair_results_dfs).reset_index(drop = True)
pair_results_df['Ne'] = pair_results_df['file1'].apply(parse_filename, get = 'Ne')
pair_results_df['Chr'] = pair_results_df['file1'].apply(parse_filename, get = 'Chr')
pair_results_df['batchname'] = pair_results_df['file1'].apply(parse_filename, get = 'Batchname')
pair_results_df.head()

,file1,file2,S,L,Srep,Patterson_f2,Hudson_fst,weir_Fst,NeiGst_unweghted,NeiGst_weghted,actual_loci,Ne,Chr,batchname
0,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,25,16,0,0.051884,0.129644,0.130773,0.066402,0.079361,16,50,99,unlinked
1,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,50,16,0,0.049222,0.163325,0.162893,0.088034,0.093908,16,50,99,unlinked
2,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,25,64,0,0.072053,0.179840,0.179426,0.102229,0.108797,64,50,99,unlinked
3,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,50,64,0,0.057215,0.147545,0.147707,0.077664,0.084640,64,50,99,unlinked
4,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,./share/unlinked/Ne-50_Chr-99/Ne-50_Chr-99_Fre...,25,256,0,0.072846,0.182930,0.183148,0.098007,0.110662,256,50,99,unlinked


In [15]:
pair_results_df.to_csv('./share/results/{}.pair.results.txt'.format(batchname), index = False, sep = '\t', na_rep = 'NA')

In [16]:
assert False

AssertionError: 

In [ ]:
import simNe_functions_analyze as simNe_anal

In [ ]:
test_genos = np.loadtxt("/home/ryan/simNe/share/july15/Ne-800_Chr-8/Ne-800_Chr-8_Frep-968.geno", dtype = 'int8', delimiter='\t')
test_genos.shape

In [ ]:
test_2 = simNe_anal.get_r2_fast(test_genos)

In [ ]:
aa = np.linalg.matrix_rank(test_2)

In [ ]:
bb = np.linalg.matrix_rank(test_genos)

In [ ]:
aa, bb

In [ ]:
test_genos.shape

In [ ]:
results_df = results_df[[
 'batchname',
 'file',
 'Ne',
 'Chr',
 'Frep',
 'L',
 'S',
 'Srep',
 'mean_r2',
 'Locus_pairs',
 'temporalF_u_subset',
 'temporalF_w_subset',
 'temporalF_u_full',
 'temporalF_w_full',
 'Pf2',
 'Hfst',
'Jorde_Ryman_F'
]]

In [ ]:
sns.lmplot(data = results_df, row = 'L', y = 'Hfst', hue = 'Ne', col = 'S', x = 'Chr', sharey = False, fit_reg = False)

In [ ]:
results_df.groupby(['Ne', 'Chr', 'L', 'S']).count()
grouped_var = results_df.groupby(['Ne', 'Chr', 'L', 'S']).var().reset_index()
grouped_mean = results_df.groupby(['Ne', 'Chr', 'L', 'S']).mean().reset_index()

#grouped_var.head()
grouped_var.head(5)

In [ ]:
g =sns.lmplot(data = grouped_var, row = 'L', y = 'temporalF_w_subset', hue = 'Ne', col = 'S', x = 'Chr', sharey = False, fit_reg = False, ci = None)
g.set(ylim=(-.00001, .000005))

In [ ]:
results_df.head()

In [ ]:
results_df['file'].apply(parse_filename, convert_dtype = True)

In [ ]:
pd.concat([pd.read_csv(xx, sep = '\t', header = 0, index_col=False) for xx in results_list])

In [ ]:
pd.read_csv('./share/mar14a/Ne-800_Chr-4/results.txt', sep = '\t', header = 0, index_col=False)

In [ ]:
# Gather results
target_dir='./share/{}/Ne-{}_Chr-{}'.format(batchname, diploid_Ne, n_chrom)